# Introduction

Using the models we have try to fill in the metadata spreadsheet & submit test records to the IGVF test portal.

This try had all the files from each platform type attached to one measurement set. But with Jennifer we decided this is probably a poor idea.

In [1]:
import bz2
from collections import Counter, namedtuple
import datetime
import numpy
import os
import pandas
from pathlib import Path
import re
from subprocess import run, PIPE
import sys
import zoneinfo

os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'mousedemo.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

import django
from django.contrib.auth import get_user_model
from django.db import DEFAULT_DB_ALIAS

MOUSEDEMO = str(Path("mousedemo").absolute())
if MOUSEDEMO not in sys.path:
    sys.path.append(MOUSEDEMO)

django.setup()

from mousedemo import settings
from igvf_mice import models

In [2]:
EC = str(Path("~/proj/encoded_client").expanduser())
if EC not in sys.path:
    sys.path.append(EC)
    
from encoded_client import encoded

In [3]:
server = encoded.ENCODED("api.sandbox.igvf.org")

validator = encoded.DCCValidator(server)

In [4]:
award = "/awards/HG012077/"
labs = ["/labs/lior-pachter/", "/labs/grant-macgregor/", "/labs/barbara-wold/", "/labs/ali-mortazavi/"]
lab = labs[-1]
jax = "/sources/jackson-labs/"
species = 'Mus musculus'

In [5]:
strain_background = {
    "A/J": "A/J (AJ)",
    "C57BL/6J": "C57BL/6J (B6)",
    "129S1/SvImJ": "129S1/SvImJ (129)",
    "NOD/ShiLtJ": "NOD/ShiLtJ (NOD)",
    "NZO/HlLtJ": "NZO/H1LtJ (NZO)",
    "CAST/EiJ": "CAST/EiJ (CAST)",
    "PWK/PhJ": "PWK/PhJ (PWK)",
    "WSB/EiJ": "WSB/EiJ (WSB)",
    #"CAST (M. m. castaneus)",
    #"WSB (M. m. domesticus)",
    #"PWK (M. m. musculus)",
}

In [6]:
test_run =  "IGVF_003"
plate_003 = models.SplitSeqPlate.objects.get(name=test_run)

In [7]:
plate_003.sublibrary_set.filter(nuclei__lt=10000)

<QuerySet [<Sublibrary: 003_8A>]>

In [8]:
tissues = {}
mice = {}
for well in plate_003.splitseqwell_set.all():
    for biosample in well.biosample.all():
        for tissue in biosample.tissue.all():
            tissues[tissue.name] = tissue
            mice[tissue.mouse.name] = tissue.mouse

# rodent_donor

In [9]:
def format_sex(value):
    if models.SexEnum.MALE == value:
        return "male"
    elif models.SexEnum.FEMALE == value:
        return "female"

def get_accession_string_or_none(record):
    accessions = record.accession.all()
    if len(accessions) == 0:
        return None
    else:
        return ",".join([x.name for x in accessions])
    
rodent_donor = []
for mouse_name in sorted(mice):
    mouse = mice[mouse_name]
    dcc_row = {
        #"#response": None,
        #"#response_time": None,
        "accession": get_accession_string_or_none(mouse),
        "uuid": None, 
        "aliases:array": f"ali-mortazavi:{mouse.name}",
        "award": award,
        "lab": lab,
        "taxa": species,
        "sex": format_sex(mouse.sex),
        "strain": mouse.strain.name,
        "references": None,
        "url": mouse.strain.url,
        "source": jax,
        "lot_id": None,
        "product_id": mouse.strain.jax_catalog_number,
        "documents": None,
        "alternate_accessions": None,
        "submitter_comment": None,
        "description": None,
        "parents": None,
        "traits": None,
        "phenotypic_features": None,
        "external_resources": None,
        "strain_background": strain_background[mouse.strain.name],
        "genotype": None,
        "individual_rodent:boolean": True,
        "rodent_identifier": mouse.name,
    }
    rodent_donor.append(dcc_row)
    
rodent_donor = pandas.DataFrame(rodent_donor)

dry_run = True
created = server.post_sheet("rodent_donor", rodent_donor, dry_run=dry_run, verbose=True, validator=validator)
if len(created) > 0 and not dry_run:
    rodent_donor.to_excel("rodent_donor.xlsx")
rodent_donor

,accession,uuid,aliases:array,award,lab,taxa,sex,strain,references,url,...,submitter_comment,description,parents,traits,phenotypic_features,external_resources,strain_background,genotype,individual_rodent:boolean,rodent_identifier
0,TSTDO36427294,None,ali-mortazavi:016_B6J_10F,/awards/HG012077/,/labs/ali-mortazavi/,Mus musculus,female,C57BL/6J,None,http://www.informatics.jax.org/inbred_strains/...,...,None,None,None,None,None,None,C57BL/6J (B6),None,True,016_B6J_10F
1,TSTDO96818159,None,ali-mortazavi:017_B6J_10M,/awards/HG012077/,/labs/ali-mortazavi/,Mus musculus,male,C57BL/6J,None,http://www.informatics.jax.org/inbred_strains/...,...,None,None,None,None,None,None,C57BL/6J (B6),None,True,017_B6J_10M
2,would create,None,ali-mortazavi:018_B6J_10F,/awards/HG012077/,/labs/ali-mortazavi/,Mus musculus,female,C57BL/6J,None,http://www.informatics.jax.org/inbred_strains/...,...,None,None,None,None,None,None,C57BL/6J (B6),None,True,018_B6J_10F
3,would create,None,ali-mortazavi:019_B6J_10M,/awards/HG012077/,/labs/ali-mortazavi/,Mus musculus,male,C57BL/6J,None,http://www.informatics.jax.org/inbred_strains/...,...,None,None,None,None,None,None,C57BL/6J (B6),None,True,019_B6J_10M
4,would create,None,ali-mortazavi:020_B6J_10F,/awards/HG012077/,/labs/ali-mortazavi/,Mus musculus,female,C57BL/6J,None,http://www.informatics.jax.org/inbred_strains/...,...,None,None,None,None,None,None,C57BL/6J (B6),None,True,020_B6J_10F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,would create,None,ali-mortazavi:090_CASTJ_10F,/awards/HG012077/,/labs/ali-mortazavi/,Mus musculus,female,CAST/EiJ,None,http://www.informatics.jax.org/inbred_strains/...,...,None,None,None,None,None,None,CAST/EiJ (CAST),None,True,090_CASTJ_10F
60,would create,None,ali-mortazavi:091_CASTJ_10M,/awards/HG012077/,/labs/ali-mortazavi/,Mus musculus,male,CAST/EiJ,None,http://www.informatics.jax.org/inbred_strains/...,...,None,None,None,None,None,None,CAST/EiJ (CAST),None,True,091_CASTJ_10M
61,would create,None,ali-mortazavi:093_CASTJ_10M,/awards/HG012077/,/labs/ali-mortazavi/,Mus musculus,male,CAST/EiJ,None,http://www.informatics.jax.org/inbred_strains/...,...,None,None,None,None,None,None,CAST/EiJ (CAST),None,True,093_CASTJ_10M
62,would create,None,ali-mortazavi:094_CASTJ_10F,/awards/HG012077/,/labs/ali-mortazavi/,Mus musculus,female,CAST/EiJ,None,http://www.informatics.jax.org/inbred_strains/...,...,None,None,None,None,None,None,CAST/EiJ (CAST),None,True,094_CASTJ_10F


# tissue

In [10]:
def get_accession_list_or_none(record, length):
    accessions = record.accession.all()
    if len(accessions) == 0:
        return [None] * length
    elif len(accessions) == length:
        return [x.name for x in accessions]
    else:
        raise ValueError("Unexpected number of accessions {} {}".format(accessions, length))

tissue_sheet = []
for tissue_name in tissues:
    tissue = tissues[tissue_name]
    donor_alias = f"ali-mortazavi:{tissue.mouse.name}"
    ontology_terms = tissue.ontology_term.all()
    accessions = get_accession_list_or_none(tissue, len(ontology_terms))
    for term, accession_id in zip(ontology_terms, accessions):
        curie = term.curie.replace(":", "_")
        alias = f"ali-mortazavi:{tissue.name}_{curie}"
        sample_term = "/sample-terms/{}/".format(curie)
        if numpy.all(pandas.isnull(accession_id)):
            accession_id = None
        dcc_row = {
            "accession": accession_id,
            "uuid": None,
            "aliases:array": alias,
            "award": award,
            "lab": lab,
            "source": jax,
            "donors:array": donor_alias,
            "taxa": species,
            "biosample_term": sample_term,
            "term_names:skip": term.name,
        }
        tissue_sheet.append(dcc_row)
    
tissue_sheet = pandas.DataFrame(tissue_sheet)

dry_run=True
created = server.post_sheet("tissue", tissue_sheet, dry_run=dry_run, verbose=True, validator=validator)
if len(created) > 0 and not dry_run:
    tissue_sheet.to_excel("tissue_sheet.xlsx")
tissue_sheet

,accession,uuid,aliases:array,award,lab,source,donors:array,taxa,biosample_term,term_names:skip
0,TSTSM69861046,None,ali-mortazavi:016_B6J_10F_03_NTR_0000646,/awards/HG012077/,/labs/ali-mortazavi/,/sources/jackson-labs/,ali-mortazavi:016_B6J_10F,Mus musculus,/sample-terms/NTR_0000646/,left cerebral cortex
1,TSTSM75831182,None,ali-mortazavi:016_B6J_10F_03_NTR_0000750,/awards/HG012077/,/labs/ali-mortazavi/,/sources/jackson-labs/,ali-mortazavi:016_B6J_10F,Mus musculus,/sample-terms/NTR_0000750/,Hippocampal formation left
2,TSTSM69542835,None,ali-mortazavi:017_B6J_10M_03_NTR_0000646,/awards/HG012077/,/labs/ali-mortazavi/,/sources/jackson-labs/,ali-mortazavi:017_B6J_10M,Mus musculus,/sample-terms/NTR_0000646/,left cerebral cortex
3,TSTSM28050193,None,ali-mortazavi:017_B6J_10M_03_NTR_0000750,/awards/HG012077/,/labs/ali-mortazavi/,/sources/jackson-labs/,ali-mortazavi:017_B6J_10M,Mus musculus,/sample-terms/NTR_0000750/,Hippocampal formation left
4,would create,None,ali-mortazavi:018_B6J_10F_03_NTR_0000646,/awards/HG012077/,/labs/ali-mortazavi/,/sources/jackson-labs/,ali-mortazavi:018_B6J_10F,Mus musculus,/sample-terms/NTR_0000646/,left cerebral cortex
...,...,...,...,...,...,...,...,...,...,...
187,would create,None,ali-mortazavi:051_NZOJ_10M_06_UBERON_0000948,/awards/HG012077/,/labs/ali-mortazavi/,/sources/jackson-labs/,ali-mortazavi:051_NZOJ_10M,Mus musculus,/sample-terms/UBERON_0000948/,heart
188,would create,None,ali-mortazavi:096_WSBJ_10F_06_UBERON_0000948,/awards/HG012077/,/labs/ali-mortazavi/,/sources/jackson-labs/,ali-mortazavi:096_WSBJ_10F,Mus musculus,/sample-terms/UBERON_0000948/,heart
189,would create,None,ali-mortazavi:052_NZOJ_10F_06_UBERON_0000948,/awards/HG012077/,/labs/ali-mortazavi/,/sources/jackson-labs/,ali-mortazavi:052_NZOJ_10F,Mus musculus,/sample-terms/UBERON_0000948/,heart
190,would create,None,ali-mortazavi:063_WSBJ_10M_06_UBERON_0000948,/awards/HG012077/,/labs/ali-mortazavi/,/sources/jackson-labs/,ali-mortazavi:063_WSBJ_10M,Mus musculus,/sample-terms/UBERON_0000948/,heart


# measurement_set

In [11]:
for run in models.SequencingRun.objects.all():
    print(run.platform.name, run.name, run.plate)

nanopore igvf_003/nanopore IGVF_003
nextseq igvf_003/nextseq IGVF_003
novaseq igvf_003/nova1 IGVF_003
novaseq igvf_003/nova2 IGVF_003
nextseq igvf_004/nextseq IGVF_004
novaseq igvf_004/nova1 IGVF_004
novaseq igvf_004/nova2 IGVF_004
nextseq igvf_005/nextseq IGVF_005
novaseq igvf_005/nova1 IGVF_005
novaseq igvf_005/nova2 IGVF_005
nextseq igvf_006/nextseq IGVF_006


In [12]:

measurement_plates = {}
for run in models.SequencingRun.objects.all():
    platform = run.platform.name
    if platform != "nanopore":
        name = "{}_{}".format(run.plate.name.lower(), run.platform.name)
        alias = f"ali-mortazavi:{name}"
        measurement_plates.setdefault(alias, set()).add(run.plate)
    
    
for alias in measurement_plates:
    assert len(measurement_plates[alias]) == 1

print(measurement_plates)

{'ali-mortazavi:igvf_003_nextseq': {<SplitSeqPlate: IGVF_003>}, 'ali-mortazavi:igvf_003_novaseq': {<SplitSeqPlate: IGVF_003>}, 'ali-mortazavi:igvf_004_nextseq': {<SplitSeqPlate: IGVF_004>}, 'ali-mortazavi:igvf_004_novaseq': {<SplitSeqPlate: IGVF_004>}, 'ali-mortazavi:igvf_005_nextseq': {<SplitSeqPlate: IGVF_005>}, 'ali-mortazavi:igvf_005_novaseq': {<SplitSeqPlate: IGVF_005>}, 'ali-mortazavi:igvf_006_nextseq': {<SplitSeqPlate: IGVF_006>}}


In [13]:
def get_samples_donors(plate):
    samples = []
    donors = {}
    for well in plate.splitseqwell_set.all():
        for biosample in well.biosample.all():
            for tissue in biosample.tissue.all():
                mouse_alias = f"ali-mortazavi:{tissue.mouse.name}"
                # I need an ordered set of donors
                donors[mouse_alias] = None
                for term in tissue.ontology_term.all():
                    curie = term.curie.replace(":", "_")
                    tissue_alias = f"ali-mortazavi:{tissue.name}_{curie}"
                    samples.append(tissue_alias)

    return {
        "samples": samples,
        "donor": list(donors.keys())
    }

In [14]:
measurements = []

for alias in measurement_plates:
    plate = next(iter(measurement_plates[alias]))
    if plate.name == test_run:
        plate_details = get_samples_donors(plate)
        
        dcc_row = {
            "accession": None,
            "uuid": None,
            "aliases:array": alias,
            "award": award,
            "lab": lab,
            "assay_term": "/assay-terms/OBI_0003109/", # single-nucleus RNA sequencing assay
            "documents": None,
            "alternate_accessions": None,
            "submitter_comment": None,
            "description": None,
            "samples:array": ",".join(plate_details["samples"]),
            #"samples_len:skip": len(plate_details["samples"]),
            #"donors:array": ",".join(plate_details["donor"]),
            #"donors_len:skip": len(plate_details["donor"]),
            "protocol": None,
        }
        measurements.append(dcc_row)
        
measurements = pandas.DataFrame(measurements)
dry_run=True
created = server.post_sheet("measurement_set", measurements, dry_run=dry_run, verbose=True, validator=validator)
if len(created) > 0 and not dry_run:
    measurements.to_excel("measurement_set.xlsx", index=False)
measurements

,accession,uuid,aliases:array,award,lab,assay_term,documents,alternate_accessions,submitter_comment,description,samples:array,protocol
0,would create,None,ali-mortazavi:igvf_003_nextseq,/awards/HG012077/,/labs/ali-mortazavi/,/assay-terms/OBI_0003109/,None,None,None,None,"ali-mortazavi:016_B6J_10F_03_NTR_0000646,ali-m...",None
1,would create,None,ali-mortazavi:igvf_003_novaseq,/awards/HG012077/,/labs/ali-mortazavi/,/assay-terms/OBI_0003109/,None,None,None,None,"ali-mortazavi:016_B6J_10F_03_NTR_0000646,ali-m...",None


# Link files to runs

In [15]:
known_fastqs = {}
with open("current_fastq_runs.txt", "rt") as instream:
    for line in instream:
        filename = Path(line.rstrip())
        known_fastqs.setdefault(filename.parts[0], {}).setdefault(filename.parts[1], []).append(filename)

print (known_fastqs.keys())

dict_keys(['igvf_004', 'igvf_005', 'igvf_003'])


In [16]:
known_md5s = {}

for md5sum_name in ["igvf_003.md5sums"]:
    with open(md5sum_name, "rt") as instream:
        run_name = md5sum_name.split(".")[0]
        for line in instream:
            records = line.split()
            md5sum = records[0]
            filename = Path(records[1])
            known_md5s.setdefault(run_name, {}).setdefault(filename.parts[0], {})[filename.name] = md5sum

In [17]:
run_groups = {
    "ali-mortazavi:igvf_003_nextseq": ["nextseq"],
    "ali-mortazavi:igvf_003_novaseq": ["nova1", "nova2"],
}

In [22]:
nextseq_re = re.compile(r"(?P<sublibrary_id>[\d]+_[\d]+[A-Z])_(?P<read>R[12])\.fastq\.gz")
#print(nextseq_re.match("003_67A_R1.fastq.gz").groups())
nova_re = re.compile(r"(?P<sublibrary_id>Sublibrary_[\d]+)_S[\d+]+_(?P<lane>L[\d]+)_(?P<read>[IR][123])_(?P<fragment>[\d]+)\.fastq\.gz")
#print(nova_re.match("Sublibrary_11_S10_L004_R1_001.fastq.gz").groups())
file_patterns = {
    "nextseq": nextseq_re,
    "nova1": nova_re,
    "nova2": nova_re
}

path_to_platform = {
    "nextseq": "nextseq",
    "nova1": "novaseq",
    "nova2": "novaseq",
}

In [29]:
def run_key(x, pattern):
    if isinstance(x, Path):
        x = x.name

    match = pattern.match(x)
    assert match is not None
    groups = match.groupdict()
    if groups["sublibrary_id"].startswith("Sublibrary"):
        sublibrary_id = int(groups["sublibrary_id"].split("_")[1])
    else:
        run, library = groups["sublibrary_id"].split("_")
        head, tail = library[:-1], library[-1:]
        sublibrary_id = (int(run), int(head), tail)
        
    if "lane" in groups:
        lane_id = int(groups["lane"][1:])
    else:
        lane_id = None
        
    read = groups["read"]

    return(sublibrary_id, lane_id, read)
        
sequencing_data = []
plate_name = "igvf_003"
run_counts = {
    "nextseq": 1,
    "novaseq": 1,
}
run_breaks = {
    "nextseq": None,
    "novaseq": None,
}
for alias in run_groups:
    for run_name in run_groups[alias]:
        pattern = file_patterns[run_name]
        
        for filename in sorted(known_fastqs[plate_name][run_name], key=lambda x: run_key(x, pattern)):
            sublibrary_id, lane_id, read = run_key(filename, pattern)
            
            if read == "I1":
                continue
            
            # Group by sublibrary_id & lane_id
            if run_breaks[path_to_platform[run_name]] is None:
                run_breaks[path_to_platform[run_name]] = (sublibrary_id, lane_id)
            elif run_breaks[path_to_platform[run_name]] != (sublibrary_id, lane_id):
                run_counts[path_to_platform[run_name]] += 1
                run_breaks[path_to_platform[run_name]] = (sublibrary_id, lane_id)

            dcc_row = {
                "accession": None,
                "uuid": None,
                #"aliases": None,
                "award": award,
                "lab": lab,
                "md5sum": known_md5s[plate_name][run_name][filename.name],
                "file_format": "fastq",
                "file_set": "ali-mortazavi:{}_{}".format(plate_name, path_to_platform[run_name]),
                "content_type": "reads",
                "sequencing_run:integer": run_counts[path_to_platform[run_name]],
                #"documents:array": None,
                #"submitter_comment": None,
                #"description": None,
                #"dbxrefs": None,
                #"derived_from": None,
                #"file_format_specifications": None,
                "submitted_file_name": filename,
                "illumina_read_type": read,
            }
            sequencing_data.append(dcc_row)
            
sequencing_data = pandas.DataFrame(sequencing_data)
sequencing_data.to_excel("sequence_data.xlsx", index=False)
created = server.post_sheet("sequence_data", sequencing_data, dry_run=True, verbose=True, validator=validator)
if len(created) > 0:
    print((len(created)))
    
sequencing_data

246


,accession,uuid,award,lab,md5sum,file_format,file_set,content_type,sequencing_run:integer,submitted_file_name,illumina_read_type
0,would create,None,/awards/HG012077/,/labs/ali-mortazavi/,bb696e63b6a8710ec8435b60ef6c81f8,fastq,ali-mortazavi:igvf_003_nextseq,reads,1,igvf_003/nextseq/003_8A_R1.fastq.gz,R1
1,would create,None,/awards/HG012077/,/labs/ali-mortazavi/,21967ba1a5a3218508961fac482a2952,fastq,ali-mortazavi:igvf_003_nextseq,reads,1,igvf_003/nextseq/003_8A_R2.fastq.gz,R2
2,would create,None,/awards/HG012077/,/labs/ali-mortazavi/,35350d53cf858267d294bce69e475c83,fastq,ali-mortazavi:igvf_003_nextseq,reads,2,igvf_003/nextseq/003_67A_R1.fastq.gz,R1
3,would create,None,/awards/HG012077/,/labs/ali-mortazavi/,00c68ff0f9e0217d422c57e8948d4bb4,fastq,ali-mortazavi:igvf_003_nextseq,reads,2,igvf_003/nextseq/003_67A_R2.fastq.gz,R2
4,would create,None,/awards/HG012077/,/labs/ali-mortazavi/,79db765143bb23e14c613c9e1339e9f7,fastq,ali-mortazavi:igvf_003_nextseq,reads,3,igvf_003/nextseq/003_67B_R1.fastq.gz,R1
...,...,...,...,...,...,...,...,...,...,...,...
241,would create,None,/awards/HG012077/,/labs/ali-mortazavi/,7358c7000d036cff3372417ee3166da7,fastq,ali-mortazavi:igvf_003_novaseq,reads,118,igvf_003/nova2/Sublibrary_16_S15_L002_R2_001.f...,R2
242,would create,None,/awards/HG012077/,/labs/ali-mortazavi/,892394a7d256d6945147353bf2a2fc35,fastq,ali-mortazavi:igvf_003_novaseq,reads,119,igvf_003/nova2/Sublibrary_16_S15_L003_R1_001.f...,R1
243,would create,None,/awards/HG012077/,/labs/ali-mortazavi/,cb32b3cbec11a08530765b9cf95c16ac,fastq,ali-mortazavi:igvf_003_novaseq,reads,119,igvf_003/nova2/Sublibrary_16_S15_L003_R2_001.f...,R2
244,would create,None,/awards/HG012077/,/labs/ali-mortazavi/,76f59bde3742e3e426f9424e1836b594,fastq,ali-mortazavi:igvf_003_novaseq,reads,120,igvf_003/nova2/Sublibrary_16_S15_L004_R1_001.f...,R1
